In [9]:
# Importing the Libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.ndimage.filters import convolve
from scipy.ndimage import gaussian_filter
from google.colab.patches import cv2_imshow


start = time.time()


# Load the image
image1 = cv2.imread('/content/image_1.jpg')
image2 = cv2.imread('/content/image_2.jpg')


# Preprocess the image
def preprocess_image(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (500, 500))
  image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  return image, image_gray


#Initialize the SIFT
sift = cv2.SIFT_create()


#Keypoint Descriptor
def compute_keypoints_descriptors(image, image_gray):

  keypoints, descriptors = sift.detectAndCompute(image_gray, None)

  keypoints_without_size = np.copy(image)
  keypoints_with_size = np.copy(image)

  cv2.drawKeypoints(image, keypoints, keypoints_without_size, color = (0, 255, 0))
  cv2.drawKeypoints(image, keypoints, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

  return keypoints, descriptors, keypoints_without_size, keypoints_with_size


# Image Preprocessing
training_image, training_gray = preprocess_image(image1)
test_image, test_gray = preprocess_image(image2)

# Display traning image and testing image
fx, plots = plt.subplots(2, 2, figsize=(10,10))

plots[0][0].set_title("Training Image")
plots[0][0].imshow(training_image)
plots[1][0].imshow(training_gray, cmap='gray')

plots[0][1].set_title("Testing Image")
plots[0][1].imshow(test_image)
plots[1][1].imshow(test_gray, cmap='gray')


# Keypoints Descriptor Computation
train_keypoints, train_descriptor, train_keypoints_without_size, train_keypoints_with_size = compute_keypoints_descriptors(training_image, training_gray)
test_keypoints, test_descriptor, test_keypoints_without_size, test_keypoints_with_size = compute_keypoints_descriptors(test_image, test_gray)

# Display image with and without keypoints size
fx, plots = plt.subplots(2, 2, figsize=(10,10))

plots[0][0].set_title("Train keypoints With Size")
plots[0][0].imshow(train_keypoints_with_size, cmap='gray')

plots[0][1].set_title("Train keypoints Without Size")
plots[0][1].imshow(train_keypoints_without_size, cmap='gray')

plots[1][0].set_title("Test keypoints With Size")
plots[1][0].imshow(test_keypoints_with_size, cmap='gray')

plots[1][1].set_title("Test keypoints Without Size")
plots[1][1].imshow(test_keypoints_without_size, cmap='gray')


# Print the number of keypoints detected in the training and test image
print("Number of Keypoints Detected In The Training Image: ", len(train_keypoints))
print("Number of Keypoints Detected In The Test Image: ", len(test_keypoints))


# Create a Brute Force Matcher object.
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = False)

# Perform the matching between the SIFT descriptors of the training image and the test image
matches = bf.match(train_descriptor, test_descriptor)

# The matches with shorter distance are the ones we want.
matches = sorted(matches, key = lambda x : x.distance)

result = cv2.drawMatches(training_image, train_keypoints, test_image, test_keypoints, matches, test_gray, flags = 2)

# Print total number of matching points between the training and test images
print("\nNumber of Matching Keypoints Between The Training and Query Images: ", len(matches))

# Display the best matching points
plt.rcParams['figure.figsize'] = [14.0, 7.0]
plt.title('Best Matching Points')
plt.imshow(result)
plt.show()

end = time.time()

runtime = end - start
print(f"Total time to run this code : {runtime}")

Output hidden; open in https://colab.research.google.com to view.